# Jaqal to IR

In this tutorial we will examine Jaqal. [Jaqal](https://www.sandia.gov/quantum/quantum-information-sciences/projects/qscout-jaqal/) (Just Another Quantum Assembly Language) is the QASM built by Sandia Labs for the [QSCOUT](https://www.sandia.gov/quantum/quantum-information-sciences/projects/qscout/) trapped-ion backend.

## JaqalPaq

We will primarily be using [JaqalPaq](https://gitlab.com/jaqal/jaqalpaq), the python package for Jaqal meta-programming. For details about JaqalPaq installation, see their [guide](https://gitlab.com/jaqal/jaqalpaq).

## Basic Usage Example

We've provided the `Sxx_circuit.jaqal` file in this repository: to confirm that your installation is valid, run the following code cell. You should see an idealized result array from the noiseless emulator: 50% |00>, 50% |11>

In [1]:
pip install pygsti

     |████████████████████████████████| 17.7 MB 7.6 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.0
    Uninstalling numpy-1.23.0:
      Successfully uninstalled numpy-1.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.22.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade numpy

  Using cached numpy-1.23.0-cp39-cp39-macosx_10_9_x86_64.whl (18.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.23.0 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.23.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import jaqalpaq
from jaqalpaq.parser import parse_jaqal_file
from jaqalpaq.emulator import run_jaqal_circuit
from jaqalpaq.generator import generate_jaqal_program

JaqalCircuitObject = parse_jaqal_file("Sxx_circuit.jaqal")
JaqalCircuitResults = run_jaqal_circuit(JaqalCircuitObject)
print(f"Probabilities: {JaqalCircuitResults.subcircuits[0].probability_by_str}")
JaqalProgram = generate_jaqal_program(JaqalCircuitObject)

/Users/hakanayaz/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ModuleNotFoundError: No module named 'pygsti.objects'

## Translating to Jaqal

Jaqal has built in capability to take circuits from a variety of widly used languages (Qiskit, Quil, pytket, Cirq) and turn them into Jaqal code which can run on QSCOUT. We will walk through the transpilation of our Bernstein-Vazirani circuit from Qiskit into `bernstein_vazirani.jaqal`.

In [ ]:
from qiskit import QuantumCircuit


s = '011'   # the hidden binary string
n = 3 # number of bits used to represent s


# We need a circuit with n qubits, plus one auxiliary qubit
# We also need n classical bits to write the output to
qc = QuantumCircuit(n+1, n)

# Put auxiliary in the minus state |->
# We leave the other n qubits in just |0>
qc.h(n)
qc.z(n)

# Apply Hadamard gates to each of the n "main" qubits (excluding the auxiliary)
# before querying the oracle
for i in range(n):
    qc.h(i)
    
# Apply barrier
qc.barrier()

# Apply the inner-product oracle
s = s[::-1] # reverse s to fit qiskit's qubit ordering
for q in range(n):
    if s[q] == '0':
        qc.i(q)
    else:
        qc.cx(q, n)
        
# Apply barrier
qc.barrier()

# Apply Hadamard gates after querying the oracle
for i in range(n):
    qc.h(i)


# Measurement
for i in range(n):
    qc.measure(i, i)

qc.draw()

## Translation to Jaqal

Translation from Qiskit to Jaqal occurs in two steps. First, the above circuit must be unrolled to QSCOUT's native gate set, composed of ion compatible instructions. Then, we can directly translate that circuit object into a Jaqal circuit object.

In [ ]:
from qiskit.transpiler import PassManager
from jaqalpaq.transpilers.qiskit import ion_pass_manager, jaqal_circuit_from_qiskit_circuit

# ion_pass_manager provides a qiskit PassManager object which is supposed to 
# unroll a qiskit circuit to Jaqal gates, however it struggles
# with Qiskit 'I' gate
ion_qc = ion_pass_manager().run(qc)


In [ ]:
# In order to deal with I, we need to add a relation to the Equivalence Library
# of a pass called 'Unroll Custom Definitions'. To get at that pass:
pm = ion_pass_manager()
passes_array_dict_array = pm.passes()
passes_array_dict = passes_array_dict_array[0]
passes_array = passes_array_dict['passes']
unroll_custom_defs = passes_array[0]
print(dir(unroll_custom_defs))



In [ ]:
# Now, we get the equivalence library and add the relation
# I_Px (Jaqal Identity gate with same duration as Px) = I
equiv_lib = unroll_custom_defs._equiv_lib
#print(dir(equiv_lib))
#print(equiv_lib.__dict__.items())
print(equiv_lib.__dict__['_base'].__dict__.items())
# equiv_lib.add_equivalence()


#pm._equiv_lib.append()
#jaq_compat = pm.run(qc)

#jaqc = jaqal_circuit_from_qiskit_circuit(jaq_compat) #qc, qiskit_to_jaq_map)
#print(generate_jaqal_program(jaqc))